# PREPROCESSING IMAGE DATA TO LINKED DATA
In this notebook, we evaluate the image assets in a session repository.
For every jpeg/png with its accompanying xmp or xmp file, an IMAGENODE metadata class is created 
that governs all the metadata of the image (paths, pose, etc.).
As ouput, the method generates RDF GRAPHS (.ttl) files

>This codebase operates on the scan2bim2.yml environment (python 3.8)

In [90]:
from rdflib import Graph, plugin
import uuid    
import importlib
import PIL.Image as PILimage
import os.path
import cv2
#IMPORT MODULES
from context import geomapi 
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

## 1. INITIALIZE SESSION

In [91]:
## INPUTS
projectPath= os.path.abspath(os.path.join(os.getcwd(), os.pardir))+"\\tests"#"D:\\Data\\2018-06 Werfopvolging Academiestraat Gent" 
sessionPath = projectPath + "\\Sample1" #"K:\Projects\2025-03 Project FWO SB Jelle\7.Data\21-11 House Maarten\RAW data\session_22-03-13 canon
sensor='CANON'
accuracy='0.1'

## 2. READ PRESENT JPEG/PNG/XMP/XML CONTAINING IMAGES + METADATA
An IMAGENODE is created per image in the session

In [93]:
path=sessionPath+"\\100_0848_0005.JPG"
img=cv2.imread(path)
graphPath=sessionPath+"\\imgGraph.ttl"
xmpPath=sessionPath+"\\IMG_2173.xmp"
node1=ImageNode(path=path, sessionPath=sessionPath, sensor=sensor, accuracy=accuracy)
node2=ImageNode(img=img, sessionPath=sessionPath, sensor=sensor, accuracy=accuracy)
# node3=ImageNode(graphPath=graphPath, sessionPath=sessionPath, sensor=sensor, accuracy=accuracy)
# node3=ImageNode(xmpPath=xmpPath, sessionPath=sessionPath, sensor=sensor, accuracy=accuracy)


In [94]:
print(node1.subject)
print(node1.geospatialTransform)

http://815ad806-e321-11ec-adc3-b07d64b65615
[51.19156891666666, 3.543635777777778, 68.342]


In [95]:
# Retrieve all image files in the session along with their xml metadata
allSessionFilePaths=ut.get_list_of_files(sessionPath) 
nodelist=[]

for path in allSessionFilePaths:        
    
    if path.endswith(".JPG") or path.endswith(".PNG"): 
        testXmpPath=path.replace('JPG','xmp')
        if testXmpPath in allSessionFilePaths:
            nodelist.append(ImageNode(path=path,xmpPath=testXmpPath,sessionPath=sessionPath, sensor=sensor, accuracy=accuracy))
        else:
            nodelist.append(ImageNode(path=path,sessionPath=sessionPath, sensor=sensor, accuracy=accuracy))

    if path.endswith(".xml"):
        list=tl.img_xml_to_nodes(path,sessionPath=sessionPath)
        if list is not None:
            nodelist.extend(list)
          
print(str(len(nodelist))+' ImageNodes are created.')
# print(str(exifCounter)+' have poses in exif data')
# print(str(xmpCounter)+' have poses in xmp data')
# print(str(xmlCounter)+' have poses in xml data (not implemented yet)')
print('Note that these nodes do not yet contain actual data but only the metadata')

3 ImageNodes are created.
Note that these nodes do not yet contain actual data but only the metadata


## 3. CREATE RDF GRAPH FOR All IMAGENODES AND EXPORT THEM TO .TTL
An RDF graph and .ttl file is created for all images in the session
(images themselves are not stored in the graph, only metadata)

In [96]:
graph=tl.nodes_to_graph(nodelist)
subjectList=[1 for node in nodelist if node.graph is not None]
graph.serialize(destination=sessionPath+"\\imgGraph.ttl", format='ttl')
print (str(len(subjectList))+' Nodes succesfully serialized in: '+sessionPath+"\\imgGraph.ttl")

3 Nodes succesfully serialized in: c:\RepoHeinder\geomapi\tests\Sample1\imgGraph.ttl


In [97]:
print(graph.serialize())

@prefix e57: <http://libe57.org#> .
@prefix exif: <http://www.w3.org/2003/12/exif/ns#> .
@prefix gom: <https://w3id.org/gom#> .
@prefix openlabel: <https://www.asam.net/index.php?eID=dumpFile&t=f&f=3876&token=413e8c85031ae64cc35cf42d0768627514868b2f#> .
@prefix v4d: <https://w3id.org/v4d/core#> .

<http://91e5b583-e321-11ec-9969-b07d64b65615> a "<class 'geomapi.imagenode.ImageNode'>" ;
    e57:geospatialTransform "[51.19158297222222, 3.543775888888889, 68.305]" ;
    exif:imageHeight "3648" ;
    exif:imageWidth "5472" ;
    exif:resolutionUnit "2" ;
    exif:xResolution "72.0" ;
    exif:yResolution "72.0" ;
    gom:coordinateSystem "geospatial-wgs84" ;
    v4d:accuracy "0.1" ;
    v4d:kwargs "{'sessionPath': 'c:\\\\RepoHeinder\\\\geomapi\\\\tests\\\\Sample1', 'sensor': 'CANON', 'accuracy': '0.1'}" ;
    v4d:path "100_0848_0003.JPG" ;
    openlabel:sensor "CANON" ;
    openlabel:timestamp "2020:09:22 16:01:29" .

<http://91e84d80-e321-11ec-8e9b-b07d64b65615> a "<class 'geomapi.imageno